In [131]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *
import os


In [132]:
spark = (
    SparkSession.builder
    .appName("Exercise6-Optimized")
    .master("local[*]")
    .config("spark.sql.shuffle.partitions", "4")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "2g")
    .getOrCreate()
)
spark


In [133]:
csv_files_path = "csv_files"
# Check if the folder exists
if not os.path.exists(csv_files_path):
    raise FileNotFoundError(f"Folder not found: {csv_files_path}")
else:
    print(f"Found CSV folder: {csv_files_path}")


Found CSV folder: csv_files


In [134]:
csv_files_path = "csv_files"
# Check if the folder exists
if not os.path.exists(csv_files_path):
    raise FileNotFoundError(f"Folder not found: {csv_files_path}")
else:
    print(f"Found CSV folder: {csv_files_path}")


Found CSV folder: csv_files


In [135]:
df = spark.read.csv("csv_files/Divvy_Trips_2019_Q4.csv",header=True,inferSchema=True)
print("Data loaded successfully!")

Data loaded successfully!


In [136]:
df.count()

704054

In [137]:
df.show()

+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
| trip_id|         start_time|           end_time|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|  usertype|gender|birthyear|
+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|25223640|2019-10-01 00:01:39|2019-10-01 00:17:20|  2215|       940.0|             20|Sheffield Ave & K...|          309|Leavitt St & Armi...|Subscriber|  Male|     1987|
|25223641|2019-10-01 00:02:16|2019-10-01 00:06:34|  6328|       258.0|             19|Throop (Loomis) S...|          241| Morgan St & Polk St|Subscriber|  Male|     1998|
|25223642|2019-10-01 00:04:32|2019-10-01 00:18:43|  3003|       850.0|             84|Milwaukee Ave & G...|          199|Wabash Ave & Gran...|Sub

In [138]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [139]:
df1 = spark.read.csv("csv_files/Divvy_Trips_2020_Q1.csv",header=True,inferSchema=True)
print("Data loaded successfully!")

Data loaded successfully!


In [140]:
df.show()

+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
| trip_id|         start_time|           end_time|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|  usertype|gender|birthyear|
+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|25223640|2019-10-01 00:01:39|2019-10-01 00:17:20|  2215|       940.0|             20|Sheffield Ave & K...|          309|Leavitt St & Armi...|Subscriber|  Male|     1987|
|25223641|2019-10-01 00:02:16|2019-10-01 00:06:34|  6328|       258.0|             19|Throop (Loomis) S...|          241| Morgan St & Polk St|Subscriber|  Male|     1998|
|25223642|2019-10-01 00:04:32|2019-10-01 00:18:43|  3003|       850.0|             84|Milwaukee Ave & G...|          199|Wabash Ave & Gran...|Sub

In [141]:
df1.show()

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+---------+-------+--------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|start_lat|start_lng|end_lat| end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+---------+-------+--------+-------------+
|EACB19130B0CDA4A|  docked_bike|2020-01-21 20:06:59|2020-01-21 20:14:30|Western Ave & Lel...|             239|Clark St & Leland...|           326|  41.9665| -87.6884|41.9671|-87.6674|       member|
|8FED874C809DC021|  docked_bike|2020-01-30 14:22:39|2020-01-30 14:26:22|Clark St & Montro...|             234|Southport Ave & I...|           318|  41.9616|  -87.666|41.9542|-87.6644|       member|
|789F3C21E

### Questions for Divvy_Trips_2019_Q4.csv file

### Q1. What are the `average` trip duration per day?

In [142]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import *

In [143]:
def average_trip_duration_per_day(df):
    df_cleaned = df.withColumn("tripduration",regexp_replace(col("tripduration"), ",", "").cast(DoubleType()).cast(IntegerType()))

    df_with_date = df_cleaned.withColumn("date",to_date(col("start_time")))

    df_avg_per_day = df_with_date.groupBy("date").agg(avg("tripduration").alias("avg_trip_duration_raw"))
    
    df_rounded = df_avg_per_day.withColumn("avg_trip_duration",round(col("avg_trip_duration_raw"), 2)).drop("avg_trip_duration_raw")

    return df_rounded


In [144]:
result = average_trip_duration_per_day(df)
result.show()

+----------+-----------------+
|      date|avg_trip_duration|
+----------+-----------------+
|2019-10-03|          1125.57|
|2019-10-10|          1312.08|
|2019-10-11|           914.22|
|2019-10-01|          1207.61|
|2019-10-04|          1353.76|
|2019-10-12|          1405.46|
|2019-10-14|          1227.73|
|2019-10-06|          2243.91|
|2019-10-07|           1007.6|
|2019-10-08|           1186.3|
|2019-10-02|            909.5|
|2019-10-05|          1698.68|
|2019-10-09|          1173.92|
|2019-10-13|          1662.24|
|2019-10-15|           930.78|
|2019-10-18|          1205.94|
|2019-10-27|           1775.2|
|2019-10-17|            899.7|
|2019-10-21|           932.98|
|2019-10-22|          1457.83|
+----------+-----------------+
only showing top 20 rows


In [145]:
result.write.mode("overwrite").option("header", "True").csv("reports/average_trips_per_day")

### Q2. How many trips were taken per day?

In [146]:
df.show()

+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
| trip_id|         start_time|           end_time|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|  usertype|gender|birthyear|
+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|25223640|2019-10-01 00:01:39|2019-10-01 00:17:20|  2215|       940.0|             20|Sheffield Ave & K...|          309|Leavitt St & Armi...|Subscriber|  Male|     1987|
|25223641|2019-10-01 00:02:16|2019-10-01 00:06:34|  6328|       258.0|             19|Throop (Loomis) S...|          241| Morgan St & Polk St|Subscriber|  Male|     1998|
|25223642|2019-10-01 00:04:32|2019-10-01 00:18:43|  3003|       850.0|             84|Milwaukee Ave & G...|          199|Wabash Ave & Gran...|Sub

In [147]:
def total_trips_per_day(df):
    df_with_date = df.withColumn("date", to_date(col("start_time")))

    df_total_per_day = df_with_date.groupBy("date") \
                        .agg(count("trip_id").alias("total_trips_per_day")) \
                        .orderBy("date")
    df_total_per_day.show()
    return df_total_per_day


In [148]:
result = total_trips_per_day(df)
result.show()

+----------+-------------------+
|      date|total_trips_per_day|
+----------+-------------------+
|2019-10-01|              18425|
|2019-10-02|               9882|
|2019-10-03|              15647|
|2019-10-04|              14570|
|2019-10-05|              10452|
|2019-10-06|              13396|
|2019-10-07|              17256|
|2019-10-08|              17537|
|2019-10-09|              17226|
|2019-10-10|              15795|
|2019-10-11|               8016|
|2019-10-12|               8702|
|2019-10-13|              10533|
|2019-10-14|              13785|
|2019-10-15|              13297|
|2019-10-16|              12886|
|2019-10-17|              13635|
|2019-10-18|              14096|
|2019-10-19|              13953|
|2019-10-20|              10419|
+----------+-------------------+
only showing top 20 rows


+----------+-------------------+
|      date|total_trips_per_day|
+----------+-------------------+
|2019-10-01|              18425|
|2019-10-02|               9882|
|2019-10-03|              15647|
|2019-10-04|              14570|
|2019-10-05|              10452|
|2019-10-06|              13396|
|2019-10-07|              17256|
|2019-10-08|              17537|
|2019-10-09|              17226|
|2019-10-10|              15795|
|2019-10-11|               8016|
|2019-10-12|               8702|
|2019-10-13|              10533|
|2019-10-14|              13785|
|2019-10-15|              13297|
|2019-10-16|              12886|
|2019-10-17|              13635|
|2019-10-18|              14096|
|2019-10-19|              13953|
|2019-10-20|              10419|
+----------+-------------------+
only showing top 20 rows


In [149]:
result.write.mode("overwrite").option("header", "True").csv("reports/total_trips_per_day")

### Q3. What was the most popular starting trip station for each month?

In [150]:
df.show()

+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
| trip_id|         start_time|           end_time|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|  usertype|gender|birthyear|
+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|25223640|2019-10-01 00:01:39|2019-10-01 00:17:20|  2215|       940.0|             20|Sheffield Ave & K...|          309|Leavitt St & Armi...|Subscriber|  Male|     1987|
|25223641|2019-10-01 00:02:16|2019-10-01 00:06:34|  6328|       258.0|             19|Throop (Loomis) S...|          241| Morgan St & Polk St|Subscriber|  Male|     1998|
|25223642|2019-10-01 00:04:32|2019-10-01 00:18:43|  3003|       850.0|             84|Milwaukee Ave & G...|          199|Wabash Ave & Gran...|Sub

In [151]:
def most_popular_station_each_month(df):
    df_month = df.withColumn("month", month(col("start_time")))
    df_count = df_month.groupBy("month", "from_station_name").agg(count("*").alias("trip_count"))
    window = Window.partitionBy("month").orderBy(col("trip_count").desc())
    df_ranked = df_count.withColumn("rank", row_number().over(window))
    df_most_popular = df_ranked.filter(col("rank") == 1).orderBy("month")
    df_most_popular.show()
    return df_most_popular

In [152]:
result = most_popular_station_each_month(df)
result.show()

+-----+-------------------+----------+----+
|month|  from_station_name|trip_count|rank|
+-----+-------------------+----------+----+
|   10|Canal St & Adams St|      6564|   1|
|   11|Canal St & Adams St|      3445|   1|
|   12|Canal St & Adams St|      2928|   1|
+-----+-------------------+----------+----+



+-----+-------------------+----------+----+
|month|  from_station_name|trip_count|rank|
+-----+-------------------+----------+----+
|   10|Canal St & Adams St|      6564|   1|
|   11|Canal St & Adams St|      3445|   1|
|   12|Canal St & Adams St|      2928|   1|
+-----+-------------------+----------+----+



In [153]:
df.show()

+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
| trip_id|         start_time|           end_time|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|  usertype|gender|birthyear|
+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|25223640|2019-10-01 00:01:39|2019-10-01 00:17:20|  2215|       940.0|             20|Sheffield Ave & K...|          309|Leavitt St & Armi...|Subscriber|  Male|     1987|
|25223641|2019-10-01 00:02:16|2019-10-01 00:06:34|  6328|       258.0|             19|Throop (Loomis) S...|          241| Morgan St & Polk St|Subscriber|  Male|     1998|
|25223642|2019-10-01 00:04:32|2019-10-01 00:18:43|  3003|       850.0|             84|Milwaukee Ave & G...|          199|Wabash Ave & Gran...|Sub

### Q4. What were the top 3 trip stations each day for the last two weeks?

Here is the logic you should use in PySpark:

Step-by-step approach

1. Clean tripduration (if required)

2. Extract date

3. Filter only last 14 days

4. Group by date + start_station

5. rank them

6. take top 3 per day

In [154]:
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window

def top_3_trip_count(df):
    df_cleaned = df.withColumn("tripduration",regexp_replace(col("tripduration"), ",", "").cast(DoubleType()).cast("int"))
    df_with_date = df_cleaned.withColumn("date", to_date(col("start_time")))
    max_date = df_with_date.agg(F.max("date")).first()[0]
    last_14 = df_with_date.filter(col("date") > F.date_sub(F.lit(max_date), 14))
    daily_counts = last_14.groupBy("date", "from_station_name").agg(F.count("*").alias("trip_count"))
    window_func = Window.partitionBy("date").orderBy(col("trip_count").desc())
    top3 = daily_counts.select('*', rank().over(window_func).alias('rank')).filter(col('rank') <= 3)
    return top3


In [155]:
result_top3 = top_3_trip_count(df)
result_top3.show()

+----------+--------------------+----------+----+
|      date|   from_station_name|trip_count|rank|
+----------+--------------------+----------+----+
|2019-12-18| Canal St & Adams St|       123|   1|
|2019-12-18|Clinton St & Madi...|       115|   2|
|2019-12-18|Clinton St & Wash...|        94|   3|
|2019-12-19| Canal St & Adams St|       133|   1|
|2019-12-19|Clinton St & Madi...|       123|   2|
|2019-12-19|Clinton St & Wash...|        95|   3|
|2019-12-20| Canal St & Adams St|       131|   1|
|2019-12-20|Clinton St & Wash...|       109|   2|
|2019-12-20|Clinton St & Madi...|        94|   3|
|2019-12-21|Streeter Dr & Gra...|        63|   1|
|2019-12-21|Kingsbury St & Ki...|        47|   2|
|2019-12-21|Wells St & Concor...|        46|   3|
|2019-12-22|      Shedd Aquarium|        87|   1|
|2019-12-22|Lake Shore Dr & M...|        79|   2|
|2019-12-22|Streeter Dr & Gra...|        70|   3|
|2019-12-23| Canal St & Adams St|       109|   1|
|2019-12-23|Clinton St & Madi...|        87|   2|


In [156]:
df.printSchema()

root
 |-- trip_id: integer (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- bikeid: integer (nullable = true)
 |-- tripduration: string (nullable = true)
 |-- from_station_id: integer (nullable = true)
 |-- from_station_name: string (nullable = true)
 |-- to_station_id: integer (nullable = true)
 |-- to_station_name: string (nullable = true)
 |-- usertype: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthyear: integer (nullable = true)



### Q5. Do `Male's` or `Female's` take longer trips on average?

In [157]:
df_result = df.filter((F.col("gender").isNotNull()))\
    .groupBy("gender") \
    .agg(F.count("*").alias("Total Counts"))

In [158]:
df_result.show()

+------+------------+
|gender|Total Counts|
+------+------------+
|Female|      151004|
|  Male|      486459|
+------+------------+



### Q6. What is the top 10 ages of those that take the longest trips, and shortest?

In [159]:
def top_10_age_long_short(df):
    df = df.withColumn("birth_date",F.to_date(F.concat(F.lit("01-01-"), F.col("birthyear")), "dd-MM-yyyy"))
    df = df.withColumn("age",floor(datediff(current_date(), col("birth_date")) / 365.25))
    df = df.withColumn("tripduration", regexp_replace(col("tripduration"), ",", "").cast("double").cast("int"))
    df = df.withColumn("date", to_date(col("start_time")))
    top10_longest = df.filter(F.col('age').isNotNull()) \
                .orderBy(F.col("tripduration").desc()) \
                .select('age', 'tripduration').limit(10)
    top10_shortest = df.filter(F.col('age').isNotNull()) \
                .orderBy(F.col("tripduration").asc()) \
                .select('age', 'tripduration').limit(10)
    return top10_longest, top10_shortest

In [160]:
result1, result2 = top_10_age_long_short(df)
result1.show()
result2.show()

+---+------------+
|age|tripduration|
+---+------------+
| 38|     6165373|
| 26|     6039942|
| 39|     5169622|
| 31|     4809091|
| 34|     4123040|
| 31|     3512685|
| 43|     3246842|
| 43|     3047069|
| 35|     2910292|
| 55|     2708185|
+---+------------+



+---+------------+
|age|tripduration|
+---+------------+
| 37|          61|
| 31|          61|
| 44|          61|
| 29|          61|
| 40|          61|
| 51|          61|
| 40|          61|
| 59|          61|
| 36|          61|
| 27|          61|
+---+------------+

